# Clustering casa_cluster_austin_only

### Notebook automatically generated from your model

Model KMeans (k=3), trained on 2019-04-09 22:17:11.

#### Generated on 2019-04-20 18:17:39.942653

Clustering
This notebook will reproduce the steps for clustering the dataset casa_cluster_austin_only.

#### Warning

The goal of this notebook is to provide an easily readable and explainable code that reproduces the main steps
of training the model. It is not complete: some of the preprocessing done by the DSS visual machine learning is not
replicated in this notebook. This notebook will not give the same results and model performance as the DSS visual machine
learning model.

Let's start with importing the required libs :

In [ ]:
import sys
import dataiku
import numpy as np
import pandas as pd
import sklearn as sk
import dataiku.core.pandasutils as pdu
from dataiku.doctor.preprocessing import PCA
from collections import defaultdict, Counter

And tune pandas display options:

In [ ]:
pd.set_option('display.width', 3000)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

#### Importing base data

The first step is to get our machine learning dataset:

In [ ]:
# We apply the preparation that you defined. You should not modify this.
preparation_steps = []
preparation_output_schema = {u'userModified': False, u'columns': [{u'type': u'string', u'name': u'@odata.context'}, {u'type': u'string', u'name': u'@odata.id'}, {u'type': u'bigint', u'name': u'BathroomsFull'}, {u'type': u'double', u'name': u'BathroomsHalf'}, {u'type': u'bigint', u'name': u'BedroomsTotal'}, {u'type': u'string', u'name': u'City'}, {u'type': u'string', u'name': u'Coordinates'}, {u'type': u'bigint', u'name': u'CoveredSpaces'}, {u'type': u'double', u'name': u'FireplacesTotal'}, {u'type': u'string', u'name': u'Flooring'}, {u'type': u'boolean', u'name': u'GarageYN'}, {u'type': u'string', u'name': u'Levels'}, {u'type': u'bigint', u'name': u'LivingArea'}, {u'type': u'boolean', u'name': u'PoolPrivateYN'}, {u'type': u'bigint', u'name': u'PostalCode'}, {u'type': u'string', u'name': u'PropertySubType'}, {u'type': u'string', u'name': u'Roof'}, {u'type': u'string', u'name': u'Sewer'}, {u'type': u'string', u'name': u'StateOrProvince'}, {u'type': u'string', u'name': u'StreetName'}, {u'downcastedToStringFromMeaning': u'LongMeaning', u'type': u'string', u'name': u'StreetNumber'}, {u'type': u'boolean', u'name': u'ViewYN'}, {u'type': u'string', u'name': u'WaterSource'}, {u'type': u'boolean', u'name': u'WaterfrontYN'}, {u'type': u'bigint', u'name': u'YearBuilt'}, {u'type': u'double', u'name': u'UrbanPercent'}, {u'type': u'double', u'name': u'MalePopPercentage'}, {u'type': u'double', u'name': u'MaxRacePopPercent'}, {u'type': u'double', u'name': u'MultiRacePopPercent'}, {u'type': u'double', u'name': u'OwnMortgagePercentage'}, {u'type': u'double', u'name': u'OwnNoMortgagePercentage'}, {u'type': u'double', u'name': u'RentersPercentage'}, {u'type': u'double', u'name': u'AvgHouseholdSize'}, {u'type': u'double', u'name': u'HouseholdsMoreThan3GensPercetage'}, {u'type': u'double', u'name': u'AdjMinGrossIncomePerReturn'}, {u'type': u'double', u'name': u'AdjListPrice'}]}

ml_dataset_handle = dataiku.Dataset('casa_cluster_austin_only')
ml_dataset_handle.set_preparation_steps(preparation_steps, preparation_output_schema)
%time ml_dataset = ml_dataset_handle.get_dataframe(limit = 100000)

print ('Base data has %i rows and %i columns' % (ml_dataset.shape[0], ml_dataset.shape[1]))
# Five first records",
ml_dataset.head(5)

#### Initial data management

The preprocessing aims at making the dataset compatible with modeling.
At the end of this step, we will have a matrix of float numbers, with no missing values.
We'll use the features and the preprocessing steps defined in Models.

Let's only keep selected features

In [ ]:
ml_dataset = ml_dataset[[u'PropertySubType', u'LivingArea', u'AvgHouseholdSize', u'GarageYN', u'WaterfrontYN', u'BedroomsTotal', u'UrbanPercent', u'Flooring', u'FireplacesTotal', u'Levels', u'AdjListPrice', u'OwnMortgagePercentage', u'MultiRacePopPercent', u'AdjMinGrossIncomePerReturn', u'BathroomsFull', u'Roof', u'ViewYN', u'RentersPercentage', u'Sewer', u'YearBuilt', u'MalePopPercentage', u'MaxRacePopPercent', u'OwnNoMortgagePercentage', u'HouseholdsMoreThan3GensPercetage', u'PoolPrivateYN', u'BathroomsHalf', u'WaterSource', u'CoveredSpaces']]

Let's first coerce categorical columns into unicode, numerical features into floats.

In [ ]:
# astype('unicode') does not work as expected

def coerce_to_unicode(x):
    if sys.version_info < (3, 0):
        if isinstance(x, str):
            return unicode(x,'utf-8')
        else:
            return unicode(x)
    else:
        return str(x)


categorical_features = [u'PropertySubType', u'GarageYN', u'WaterfrontYN', u'Flooring', u'Levels', u'Roof', u'ViewYN', u'Sewer', u'PoolPrivateYN', u'WaterSource']
numerical_features = [u'LivingArea', u'AvgHouseholdSize', u'BedroomsTotal', u'UrbanPercent', u'FireplacesTotal', u'AdjListPrice', u'OwnMortgagePercentage', u'MultiRacePopPercent', u'AdjMinGrossIncomePerReturn', u'BathroomsFull', u'RentersPercentage', u'YearBuilt', u'MalePopPercentage', u'MaxRacePopPercent', u'OwnNoMortgagePercentage', u'HouseholdsMoreThan3GensPercetage', u'BathroomsHalf', u'CoveredSpaces']
text_features = []
from dataiku.doctor.utils import datetime_to_epoch
for feature in categorical_features:
    ml_dataset[feature] = ml_dataset[feature].apply(coerce_to_unicode)
for feature in text_features:
    ml_dataset[feature] = ml_dataset[feature].apply(coerce_to_unicode)
for feature in numerical_features:
    if ml_dataset[feature].dtype == np.dtype('M8[ns]'):
        ml_dataset[feature] = datetime_to_epoch(ml_dataset[feature])
    else:
        ml_dataset[feature] = ml_dataset[feature].astype('double')

Let's copy our dataset to keep it for eventual profiling at the end.

In [ ]:
# train dataset will be the one on which we will apply ml technics
train = ml_dataset.copy()

#### Features preprocessing

The first thing to do at the features level is to handle the missing values.
Let's reuse the settings defined in the model

In [ ]:
drop_rows_when_missing = []
impute_when_missing = [{'impute_with': u'MEAN', 'feature': u'LivingArea'}, {'impute_with': u'MEAN', 'feature': u'AvgHouseholdSize'}, {'impute_with': u'MEAN', 'feature': u'BedroomsTotal'}, {'impute_with': u'MEAN', 'feature': u'UrbanPercent'}, {'impute_with': u'MEAN', 'feature': u'FireplacesTotal'}, {'impute_with': u'MEAN', 'feature': u'AdjListPrice'}, {'impute_with': u'MEAN', 'feature': u'OwnMortgagePercentage'}, {'impute_with': u'MEAN', 'feature': u'MultiRacePopPercent'}, {'impute_with': u'MEAN', 'feature': u'AdjMinGrossIncomePerReturn'}, {'impute_with': u'MEAN', 'feature': u'BathroomsFull'}, {'impute_with': u'MEAN', 'feature': u'RentersPercentage'}, {'impute_with': u'MEAN', 'feature': u'YearBuilt'}, {'impute_with': u'MEAN', 'feature': u'MalePopPercentage'}, {'impute_with': u'MEAN', 'feature': u'MaxRacePopPercent'}, {'impute_with': u'MEAN', 'feature': u'OwnNoMortgagePercentage'}, {'impute_with': u'MEAN', 'feature': u'HouseholdsMoreThan3GensPercetage'}, {'impute_with': u'MEAN', 'feature': u'BathroomsHalf'}, {'impute_with': u'MEAN', 'feature': u'CoveredSpaces'}]

# Features for which we drop rows with missing values"
for feature in drop_rows_when_missing:
    train = train[train[feature].notnull()]
    
    print ('Dropped missing records in %s' % feature)

# Features for which we impute missing values"
for feature in impute_when_missing:
    if feature['impute_with'] == 'MEAN':
        v = train[feature['feature']].mean()
    elif feature['impute_with'] == 'MEDIAN':
        v = train[feature['feature']].median()
    elif feature['impute_with'] == 'CREATE_CATEGORY':
        v = 'NULL_CATEGORY'
    elif feature['impute_with'] == 'MODE':
        v = train[feature['feature']].value_counts().index[0]
    elif feature['impute_with'] == 'CONSTANT':
        v = feature['value']
    train[feature['feature']] = train[feature['feature']].fillna(v)
    
    print ('Imputed missing values in feature %s with value %s' % (feature['feature'], coerce_to_unicode(v)))

We can now handle the categorical features (still using the settings defined in Models):

Let's dummy-encode the following features.
A binary column is created for each of the 100 most frequent values.

In [ ]:
LIMIT_DUMMIES = 100

categorical_to_dummy_encode = [u'PropertySubType', u'GarageYN', u'WaterfrontYN', u'Flooring', u'Levels', u'Roof', u'ViewYN', u'Sewer', u'PoolPrivateYN', u'WaterSource']

# Only keep the top 100 values
def select_dummy_values(train, features):
    dummy_values = {}
    for feature in categorical_to_dummy_encode:
        values = [
            value
            for (value, _) in Counter(train[feature]).most_common(LIMIT_DUMMIES)
        ]
        dummy_values[feature] = values
    return dummy_values

DUMMY_VALUES = select_dummy_values(train, categorical_to_dummy_encode)

def dummy_encode_dataframe(df):
    for (feature, dummy_values) in DUMMY_VALUES.items():
        for dummy_value in dummy_values:
            dummy_name = u'%s_value_%s' % (feature, coerce_to_unicode(dummy_value))
            df[dummy_name] = (df[feature] == dummy_value).astype(float)
        del df[feature]
        print ('Dummy-encoded feature %s' % feature)

dummy_encode_dataframe(train)

Let's rescale numerical features

In [ ]:
rescale_features = {u'BathroomsHalf': u'AVGSTD', u'OwnMortgagePercentage': u'AVGSTD', u'MaxRacePopPercent': u'AVGSTD', u'LivingArea': u'AVGSTD', u'AdjListPrice': u'AVGSTD', u'BathroomsFull': u'AVGSTD', u'OwnNoMortgagePercentage': u'AVGSTD', u'FireplacesTotal': u'AVGSTD', u'HouseholdsMoreThan3GensPercetage': u'AVGSTD', u'AvgHouseholdSize': u'AVGSTD', u'RentersPercentage': u'AVGSTD', u'AdjMinGrossIncomePerReturn': u'AVGSTD', u'YearBuilt': u'AVGSTD', u'MalePopPercentage': u'AVGSTD', u'CoveredSpaces': u'AVGSTD', u'UrbanPercent': u'AVGSTD', u'BedroomsTotal': u'AVGSTD', u'MultiRacePopPercent': u'AVGSTD'}
for (feature_name, rescale_method) in rescale_features.items():
    if rescale_method == 'MINMAX':
        _min = train[feature_name].min()
        _max = train[feature_name].max()
        scale = _max - _min
        shift = _min
    else:
        shift = train[feature_name].mean()
        scale = train[feature_name].std()
    if scale == 0.:
        del train[feature_name]
        
        print ('Feature %s was dropped because it has no variance' % feature_name)
    else:
        print ('Rescaled %s' % feature_name)
        train[feature_name] = (train[feature_name] - shift).astype(np.float64) / scale

Removing outliers

In [ ]:
# Remove outliers from train set
from dataiku.doctor.preprocessing.dataframe_preprocessing import detect_outliers

outliers = detect_outliers(train, 0.9, 100, 0.01)
train = train[~outliers]

print ("%s outliers found" % (outliers.sum()))

#### Modeling

In [ ]:
from sklearn.cluster import KMeans
clustering_model = KMeans(n_clusters=3)

We can finally cluster our dataset!

In [ ]:
%time clusters = clustering_model.fit_predict(train)

Build up our result dataset

#### Results

Inertia

In [ ]:
print (clustering_model.inertia_)

Silhouette

In [ ]:
from sklearn.metrics import silhouette_score
silhouette = silhouette_score(train.values, clusters, metric='euclidean', sample_size=2000)
print ("Silhouette score :", silhouette)

Join our original dataset with the cluster labels we found.

In [ ]:
final = train.join(pd.Series(clusters, index=train.index, name='cluster'))
final['cluster'] = final['cluster'].map(lambda cluster_id: 'cluster' + str(cluster_id))

Compute the cluster sizes

In [ ]:
size = pd.DataFrame({'size': final['cluster'].value_counts()})
size.head()

Draw a nice scatter plot

In [ ]:
axis_x = train.columns[0]   # change me
axis_y = train.columns[1]  # change me

from ggplot import ggplot, aes, geom_point
print(ggplot(aes(axis_x, axis_y, colour='cluster'), final) + geom_point())

That's it. It's now up to you to tune your preprocessing, your algo, and your analysis !
